In [1]:
import pyspark

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [5]:
spark=SparkSession \
 .builder \
 .master("local[*]") \
 .appName("test") \
 .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/28 01:00:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark

In [40]:
df_green=spark.read.parquet("../code/data/pq/green/*/*")

In [41]:
df_yellow=spark.read.parquet("../code/data/pq/yellow/*/*")

In [44]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

In [43]:
df_green=df_green \
 .withColumnRenamed("lpep_pickup_datetime","pickup_datetime") \
 .withColumnRenamed("lpep_dropoff_datetime","dropoff_datetime")

In [45]:
df_yellow=df_yellow \
 .withColumnRenamed("tpep_pickup_datetime","pickup_datetime") \
 .withColumnRenamed("tpep_dropoff_datetime","dropoff_datetime")

In [47]:
common_columns=[]

yellow_columns=df_yellow.columns
for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)
common_columns
        

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [50]:
df_green_sel=df_green \
 .select(common_columns) \
 .withColumn("service_type",F.lit("green"))

In [51]:
df_yellow_sel=df_yellow \
 .select(common_columns) \
 .withColumn("service_type",F.lit("yellow"))

In [52]:
df_trips_data=df_green_sel.unionAll(df_yellow_sel)

In [54]:
df_trips_data.groupby("service_type").count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [55]:
df_trips_data.registerTempTable("trips_data")

/home/murat/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [57]:
spark.sql("""
SELECT service_type,COUNT(1)
FROM trips_data
GROUP BY service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [58]:
df_result=spark.sql("""
SELECT 
    -- Revenue grouping 
    PULocationID as zone,
    date_trunc("month",pickup_datetime) as revenue_month,
    service_type,

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance

FROM trips_data
GROUP BY 1,2,3

""")

In [61]:
df_result.coalesce(1).write.parquet('../code/data/report/revenue/', mode='overwrite')